## Test Bench
This  notebook is used to evaluate the performance of a compression algorithm.

It is written such that a new compression algorithm can be added without altering the bench (just change the input in the config).

In [1]:
# ADD NEW ALGORITHMS HERE
from alg_nocomp import NoCompression

# CONFIG
ALG = NoCompression()
DATASET_PATH = "data/lund_building_highway.json"
NBR_ITER = 10

COMPR_PATH = "data/testbench_compressed"
DECOMPR_PATH = "data/testbench_decompressed"

In [2]:
import time
import os
from enum import Enum
import pandas as pd
from datetime import datetime
import filecmp
import random
#random.seed("abc123")

ET = Enum('ET', ['COMP_TIME', 'DECOMP_TIME', 'COMP_SIZE', 'DECOMP_SIZE',
    'VERTICES', 'TYPE', 'VERTEX_COUNT', 'AREA', 'LENGTH'
]) # EVAL_TYPE

### Benchmarking
Run the compression, decompression, along with the operations, and store the measurements. Done NBR_ITER times to average.

In [6]:
res = {x: [] for x in ET}
for i in range(NBR_ITER):

    # Compress
    s = time.perf_counter() # Used for timing
    ALG.compress(DATASET_PATH, COMPR_PATH)
    e = time.perf_counter()
    res[ET.COMP_TIME].append(e - s) # Store delta time
    res[ET.COMP_SIZE].append(os.stat(COMPR_PATH).st_size) # Store compressed file size

    # Decompress
    s = time.perf_counter()
    ALG.decompress(COMPR_PATH, DECOMPR_PATH)
    e = time.perf_counter()
    res[ET.DECOMP_TIME].append(e - s) # Store delta time
    res[ET.DECOMP_SIZE].append(os.stat(DECOMPR_PATH).st_size) # Store decompressed file size

    max_idx = ALG.geometry_count() - 1
# ------------ UNARY ------------ #
    # Get all vertices for Shape
    idx = random.randint(0, max_idx)
    s = time.perf_counter()
    ALG.vertices(idx)
    e = time.perf_counter()
    res[ET.VERTICES].append(e - s)

    # Get the type of the geometry
    idx = random.randint(0, max_idx)
    s = time.perf_counter()
    ALG.type(idx)
    e = time.perf_counter()
    res[ET.TYPE].append(e - s)

    # Get the number of vertices for a geometry
    idx = random.randint(0, max_idx)
    s = time.perf_counter()
    ALG.vertex_count(idx)
    e = time.perf_counter()
    res[ET.VERTEX_COUNT].append(e - s)

    # Get the area of the polygon, 0 otherwise
    idx = random.randint(0, max_idx)
    s = time.perf_counter()
    ALG.area(idx)
    e = time.perf_counter()
    res[ET.AREA].append(e - s)

    # Get the length of the path
    idx = random.randint(0, max_idx)
    s = time.perf_counter()
    ALG.length(idx)
    e = time.perf_counter()
    res[ET.LENGTH].append(e - s)

df = pd.DataFrame(res)
df.loc['mean'] = df.mean()
print("Geometry count:", max_idx + 1)
df

Geometry count: 16491


,ET.COMP_TIME,ET.DECOMP_TIME,ET.COMP_SIZE,ET.DECOMP_SIZE,ET.VERTICES,ET.TYPE,ET.VERTEX_COUNT,ET.AREA,ET.LENGTH
0,0.018117,0.007953,5761506.0,5761506.0,0.217489,0.180873,0.167013,9.789983e-07,5.059992e-07
1,0.005196,0.005437,5761506.0,5761506.0,0.211604,0.180032,0.168346,9.200012e-07,5.080001e-07
2,0.004938,0.005792,5761506.0,5761506.0,0.202254,0.168924,0.158748,9.500000e-07,4.799986e-07
3,0.004841,0.004776,5761506.0,5761506.0,0.195855,0.165224,0.158259,1.221000e-06,5.680013e-07
4,0.005046,0.004751,5761506.0,5761506.0,0.171329,0.171170,0.157203,1.053002e-06,5.209986e-07
5,0.005067,0.005303,5761506.0,5761506.0,0.168375,0.164288,0.153857,1.112003e-06,5.310030e-07
6,0.004626,0.005282,5761506.0,5761506.0,0.163115,0.158418,0.203852,1.130000e-06,6.070004e-07
7,0.004619,0.005431,5761506.0,5761506.0,0.163735,0.154823,0.200461,9.979995e-07,4.690010e-07
8,0.004542,0.005304,5761506.0,5761506.0,0.159509,0.155502,0.198493,8.700008e-07,4.409994e-07
9,0.005204,0.004555,5761506.0,5761506.0,0.158308,0.204953,0.171081,9.460018e-07,4.209978e-07


### Export to CSV

In [4]:
# Exports in the format: 2023-02-13_17:09:12_NoCompression.
df.to_csv("reports/" + datetime.now().strftime("%Y-%m-%d_%H:%M:%S") + "_" + ALG.__class__.__name__ + ".csv")

### Verify Integrity
Need to check that the operations are correct and that the decompression is lossless.

In [5]:
# Is decompressed file equal to original?
assert(filecmp.cmp(DECOMPR_PATH, DATASET_PATH, shallow=False))